In [1]:
import sys

In [2]:
sys.path.append("/Users/dikien/Downloads/Passage/")

In [3]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from passage.models import RNN
from passage.updates import Adadelta
from passage.layers import Embedding, GatedRecurrent, Dense
from passage.preprocessing import Tokenizer
from nltk.stem.porter import *

In [4]:
# Load the training file
train = pd.read_csv('/Users/dikien/Downloads/Search Results Relevance/train.csv')
test = pd.read_csv('/Users/dikien/Downloads/Search Results Relevance/test.csv')

In [5]:
# we dont need ID columns
idx = test.id.values.astype(int)
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [6]:
train.head(3)

,query,product_title,product_description,median_relevance,relevance_variance
0,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471


In [7]:
test.head(3)

,query,product_title,product_description
0,electric griddle,Star-Max 48 in Electric Griddle,NaN
1,phillips coffee maker,Philips SENSEO HD7810 WHITE Single Serve Pod C...,NaN
2,san francisco 49ers,2013 San Francisco 49ers Clock,A 2013 San Francisco 49ers clock is the ultima...


In [8]:
# product_description은 없는 것이 있음
print train.isnull().any(0)
print "=" * 100
print test.isnull().any(0)

query                  False
product_title          False
product_description     True
median_relevance       False
relevance_variance     False
dtype: bool
query                  False
product_title          False
product_description     True
dtype: bool


In [9]:
# non values를 " "으로 채우기
train = train.fillna(" ")
test  = test.fillna(" ")

In [10]:
# product_description은 없는 것이 있음
print train.isnull().any(0)
print "=" * 100
print test.isnull().any(0)

query                  False
product_title          False
product_description    False
median_relevance       False
relevance_variance     False
dtype: bool
query                  False
product_title          False
product_description    False
dtype: bool


In [11]:
#remove html, remove non text or numeric, make query and title unique features for counts using prefix (accounted for in stopwords tweak)
stemmer = PorterStemmer()

## Stemming functionality
class stemmerUtility(object):
    """Stemming functionality"""
    @staticmethod
    def stemPorter(review_text):
        porter = PorterStemmer()
        preprocessed_docs = []
        for doc in review_text:
            final_doc = []
            for word in doc:
                final_doc.append(porter.stem(word))
                #final_doc.append(wordnet.lemmatize(word)) #note that lemmatize() can also takes part of speech as an argument!
            preprocessed_docs.append(final_doc)
        return preprocessed_docs

def clean(text):
#     text = html.fromstring(text).text_content().lower().strip()
    text = BeautifulSoup(text).get_text(" ")
    text = re.sub("[^a-zA-Z0-9]"," ", text)
    text = (" ").join([stemmer.stem(z) for z in text.split()])
    return text

In [12]:
print clean("jongwon . , kim ")

jongwon kim


In [13]:
# clean data
train['query'] = train['query'].apply(func=clean)
train['product_title'] = train['product_title'].apply(func=clean)
train['product_description'] = train['product_description'].apply(func=clean)

test['query'] = test['query'].apply(func=clean)
test['product_title'] = test['product_title'].apply(func=clean)
test['product_description'] = test['product_description'].apply(func=clean)

/Users/dikien/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65497012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Users/dikien/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65516012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP 

In [14]:
def merge_rows(x):
    query = x[0]
    product_title = x[1]
    product_description  = x[2]
    return query + ' ' + product_title + ' ' + product_description

In [15]:
trainX = train[['query', 'product_title', 'product_description']].apply(func=merge_rows, axis=1)
testX = test[['query', 'product_title', 'product_description']].apply(func=merge_rows, axis=1)

In [16]:
trainY = train["median_relevance"]

In [17]:
tokenizer = Tokenizer(min_df=10, max_features=100000)
trainX = tokenizer.fit_transform(trainX)

print("Training data tokenized.")

Training data tokenized.


In [18]:
tokenizer.n_features

4354

In [19]:
layers = [
    Embedding(size=256, n_features=tokenizer.n_features),
    GatedRecurrent(size=512, activation='tanh', gate_activation='steeper_sigmoid', init='orthogonal', seq_output=False, p_drop=0.75),
    Dense(size=1, activation='sigmoid', init='orthogonal')
]

model = RNN(layers=layers, cost='bce', updater=Adadelta(lr=0.5))
model.fit(trainX, trainY, n_epochs=1)

Epoch 0 Seen 10058 samples Avg cost -842.1429 Time elapsed 410 seconds


/Users/dikien/anaconda/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


[array(0.7019436132583434),
 array(0.5191049687834116),
 array(0.29999605990878175),
 array(-0.04111086707732512),
 array(-1.0811351469475654),
 array(-20.40324655473114),
 array(-39.811147414137515),
 array(-60.45484734692913),
 array(-73.62591611849797),
 array(-82.85591777530604),
 array(-96.85875457925671),
 array(-103.88725965248537),
 array(-112.82843714719404),
 array(-107.05845253427695),
 array(-126.97924800622309),
 array(-139.61813563132978),
 array(-161.1675951836773),
 array(-171.79844654248967),
 array(-190.4508372990546),
 array(-187.95761881580418),
 array(-184.3901830466014),
 array(-202.17437268631667),
 array(-222.62368797243573),
 array(-216.37605081942294),
 array(-239.14372925180808),
 array(-246.33164539030076),
 array(-258.456589615913),
 array(-257.84030395989174),
 array(-265.5873604341535),
 array(-307.0250508323931),
 array(-312.59875878435895),
 array(-326.0943530150654),
 array(-350.9150553183064),
 array(-349.50489736275216),
 array(-394.25019074602704),


In [32]:
print model.layers[0].__dict__

{'settings': {'init': 'uniform', 'weights': None, 'n_features': 4354, 'size': 256}, 'wv': <TensorType(float64, matrix)>, 'init': <function uniform at 0x109a86a28>, 'params': [<TensorType(float64, matrix)>], 'input': <TensorType(int32, matrix)>, 'n_features': 4354, 'size': 256}


In [33]:
testX = tokenizer.transform(testX)
prediction = model.predict(testX)

In [43]:
prediction = prediction.astype('int32')

In [44]:
# Create your first submission file
submission = pd.DataFrame({"id": idx, "prediction": prediction.ravel()})
submission.to_csv("submission.csv", index=False)